# 🔗 Chains: Conectando os Pontos da IA - Do Simples ao Complexo!

**Por Pedro Nunes Guth - Módulo 4 de 15 do Curso LangChain v0.2**

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-04_img_01.png)

Bora entender como transformar componentes isolados em verdadeiras máquinas de processamento inteligente! 🚀

## 🎯 O que São Chains?

Tá, mas o que é uma Chain? Imagina que você tá fazendo um prato de macarronada:

1. Primeiro você ferve a água 💧
2. Depois cozinha o macarrão 🍝
3. Então prepara o molho 🍅
4. Por fim, junta tudo e serve! 🍽️

Uma **Chain** é exatamente isso: uma sequência de operações que acontecem uma após a outra, onde o resultado de uma etapa vira entrada da próxima!

### Por que Chains são Importantes?

- **Modularidade**: Cada componente tem sua função específica
- **Reutilização**: Você pode usar os mesmos blocos em diferentes chains
- **Simplicidade**: Quebra problemas complexos em partes menores
- **Manutenção**: Facilita debuggar e melhorar cada etapa

**🎯 Dica do Pedro**: Lembra dos módulos anteriores? Vamos conectar PromptTemplates + ChatModels + OutputParsers em chains poderosas!

In [ ]:
# Setup inicial - Instalando e importando tudo que precisamos
!pip install langchain langchain-google-genai python-dotenv matplotlib seaborn -q

import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, Any

# Configuração para gráficos mais bonitos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🔧 Setup inicial concluído! Bora começar a brincadeira!")

In [ ]:
# Carregando variáveis de ambiente e configurando o modelo
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# Configurando nosso modelo (lembra do módulo 2?)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

print("🤖 Modelo configurado e pronto para usar!")
print(f"📊 Modelo: {llm.model_name}")
print(f"🌡️ Temperatura: {llm.temperature}")

## 🧱 Anatomia de uma Chain Simples

Vamos começar com o básico! Uma chain simples conecta:

$$\text{Input} \rightarrow \text{PromptTemplate} \rightarrow \text{LLM} \rightarrow \text{OutputParser} \rightarrow \text{Output}$$

Essa é a **fórmula mágica** que vamos usar em 90% dos nossos casos!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-04_img_02.png)

In [ ]:
# Criando nossa primeira chain simples
# Lembra do módulo 3? Vamos usar PromptTemplate + OutputParser!

# 1. Definindo o template do prompt
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um chef especialista em culinária brasileira."),
    ("human", "Me sugira uma receita com {ingrediente} que seja {tipo_prato}")
])

# 2. Definindo o parser de saída
output_parser = StrOutputParser()

# 3. Criando a chain usando LCEL (Langchain Expression Language)
# Lembra do módulo 2? O operador | conecta os componentes!
receita_chain = prompt_template | llm | output_parser

print("🔗 Chain criada com sucesso!")
print("📋 Componentes: PromptTemplate → LLM → OutputParser")

In [ ]:
# Testando nossa primeira chain

resultado = receita_chain.invoke({
    "ingrediente": "batata doce",
    "tipo_prato": "sobremesa"
})

print("🍽️ RESULTADO DA CHAIN:")
print("=" * 50)
print(resultado)
print("=" * 50)
print("\n✨ Liiindo! A chain funcionou perfeitamente!")

## 🔄 Tipos de Chains no LangChain

Existem várias "receitas prontas" de chains que podemos usar. É como ter um livro de receitas da vovó!

### 1. **SimpleSequentialChain**
- Output de uma vira input da próxima
- Linear e direto

### 2. **SequentialChain** 
- Múltiplos inputs e outputs
- Mais flexível

### 3. **Custom Chains (LCEL)**
- Você monta do seu jeito
- Total controle!

**🎯 Dica do Pedro**: Na v0.2, o LCEL é rei! É mais flexível e poderoso que as chains prontas.

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-04_img_03.png)

In [ ]:
# Criando uma Sequential Chain mais complexa
# Vamos criar um sistema que: analisa → resume → traduz

# Chain 1: Análise de sentimento
analise_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um especialista em análise de sentimentos."),
    ("human", "Analise o sentimento do seguinte texto e classifique como positivo, negativo ou neutro: {texto}")
])

analise_chain = analise_template | llm | StrOutputParser()

# Chain 2: Resumo
resumo_template = ChatPromptTemplate.from_messages([
    ("system", "Você é especialista em resumos concisos."),
    ("human", "Resuma esta análise em uma frase: {analise}")
])

resumo_chain = resumo_template | llm | StrOutputParser()

print("🔗 Chains individuais criadas!")
print("📊 Chain 1: Análise de Sentimento")
print("📝 Chain 2: Resumo")

In [ ]:
# Agora vamos conectar as chains usando LCEL!
# Aqui está a mágica: o resultado de uma vira entrada da outra

def criar_sequential_chain():
    """Cria uma chain sequencial que processa texto em etapas"""
    
    # Usando RunnablePassthrough para manter o contexto
    chain_completa = (
        # Etapa 1: Análise
        {"analise": analise_chain, "texto_original": RunnablePassthrough()}
        # Etapa 2: Resumo (usa o resultado da análise)
        | {"resumo": resumo_chain, "analise_completa": lambda x: x["analise"], "texto": lambda x: x["texto_original"]}
    )
    
    return chain_completa

# Criando a chain completa
chain_completa = criar_sequential_chain()

print("🎯 Chain sequencial criada!")
print("🔄 Fluxo: Texto → Análise → Resumo")

In [ ]:
# Testando a chain sequencial

texto_teste = """
Hoje foi um dia incrível! Consegui finalizar o projeto que estava trabalhando há semanas.
A equipe toda se empenhou muito e o resultado ficou além das expectativas.
Estou muito orgulhoso do que conseguimos construir juntos!
"""

resultado_completo = chain_completa.invoke({"texto": texto_teste.strip()})

print("📊 RESULTADO DA CHAIN SEQUENCIAL:")
print("=" * 60)
print(f"📝 Texto Original: {resultado_completo['texto']['texto'][:100]}...")
print(f"\n🔍 Análise Completa: {resultado_completo['analise_completa']}")
print(f"\n📋 Resumo Final: {resultado_completo['resumo']}")
print("=" * 60)
print("\n✨ Liiindo! Processamento em múltiplas etapas funcionando!")

## 📊 Visualizando o Fluxo das Chains

Nada melhor que uma visualização para entender como as chains funcionam!

In [ ]:
# Criando uma visualização do fluxo de uma chain
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import FancyBboxPatch

def visualizar_chain_flow():
    """Visualiza o fluxo de uma chain sequencial"""
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Definindo posições dos componentes
    componentes = [
        {"nome": "Input\nTexto", "pos": (1, 4), "cor": "lightblue"},
        {"nome": "Prompt\nTemplate", "pos": (3, 4), "cor": "lightgreen"},
        {"nome": "LLM\n(Gemini)", "pos": (5, 4), "cor": "orange"},
        {"nome": "Análise de\nSentimento", "pos": (7, 4), "cor": "lightcoral"},
        {"nome": "Prompt\nResumo", "pos": (9, 4), "cor": "lightgreen"},
        {"nome": "LLM\n(Gemini)", "pos": (11, 4), "cor": "orange"},
        {"nome": "Output\nFinal", "pos": (13, 4), "cor": "gold"}
    ]
    
    # Desenhando os componentes
    for i, comp in enumerate(componentes):
        # Criando caixa fancy
        box = FancyBboxPatch(
            (comp["pos"][0]-0.6, comp["pos"][1]-0.4),
            1.2, 0.8,
            boxstyle="round,pad=0.1",
            facecolor=comp["cor"],
            edgecolor="black",
            linewidth=2
        )
        ax.add_patch(box)
        
        # Adicionando texto
        ax.text(comp["pos"][0], comp["pos"][1], comp["nome"], 
                ha="center", va="center", fontsize=10, fontweight="bold")
        
        # Adicionando setas
        if i < len(componentes) - 1:
            ax.arrow(comp["pos"][0] + 0.6, comp["pos"][1], 0.8, 0,
                    head_width=0.15, head_length=0.2, fc="darkblue", ec="darkblue")
    
    # Configurações do gráfico
    ax.set_xlim(0, 14)
    ax.set_ylim(2, 6)
    ax.set_title("🔗 Fluxo de uma Chain Sequencial", fontsize=16, fontweight="bold", pad=20)
    ax.axis("off")
    
    # Adicionando legenda
    ax.text(7, 2.5, "Cada componente processa e passa o resultado para o próximo", 
            ha="center", fontsize=12, style="italic")
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Visualização criada! Cada seta representa o fluxo de dados.")

visualizar_chain_flow()

## 🔀 Chains Paralelas e Condicionais

Nem tudo na vida é linear! Às vezes precisamos processar coisas em paralelo ou tomar decisões.

### Processamento Paralelo
Imagina que você quer:
- Analisar sentimento **E** extrair palavras-chave **AO MESMO TEMPO**
- Depois combinar os resultados

### Chains Condicionais  
"Se o sentimento for negativo, faça X, senão faça Y"

**🎯 Dica do Pedro**: Isso vai ser crucial para o módulo de Agents que vem aí!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-04_img_04.png)

In [ ]:
# Criando chains paralelas
# Vamos analisar um texto em múltiplas dimensões simultaneamente

# Chain para análise de sentimento
sentimento_prompt = ChatPromptTemplate.from_messages([
    ("system", "Analise apenas o sentimento: positivo, negativo ou neutro."),
    ("human", "{texto}")
])
sentimento_chain = sentimento_prompt | llm | StrOutputParser()

# Chain para extração de palavras-chave
keywords_prompt = ChatPromptTemplate.from_messages([
    ("system", "Extraia as 5 palavras-chave mais importantes do texto."),
    ("human", "{texto}")
])
keywords_chain = keywords_prompt | llm | StrOutputParser()

# Chain para análise de tópicos
topicos_prompt = ChatPromptTemplate.from_messages([
    ("system", "Identifique o tópico principal do texto em uma palavra."),
    ("human", "{texto}")
])
topicos_chain = topicos_prompt | llm | StrOutputParser()

# Combinando em uma chain paralela usando dict
chain_paralela = {
    "sentimento": sentimento_chain,
    "palavras_chave": keywords_chain,
    "topico": topicos_chain,
    "texto_original": RunnablePassthrough()
}

print("⚡ Chain paralela criada!")
print("🔄 Processamento simultâneo de:")
print("   • Análise de sentimento")
print("   • Extração de palavras-chave")
print("   • Identificação de tópicos")

In [ ]:
# Testando a chain paralela

texto_teste = """
A inteligência artificial está revolucionando a forma como trabalhamos.
Ferramentas como ChatGPT e Gemini estão democratizando o acesso à tecnologia avançada.
Isso é incrível para produtividade, mas também traz desafios éticos importantes.
"""

# Executando processamento paralelo
import time
start_time = time.time()

resultado_paralelo = chain_paralela.invoke({"texto": texto_teste.strip()})

end_time = time.time()

print("⚡ RESULTADO DO PROCESSAMENTO PARALELO:")
print("=" * 60)
print(f"😊 Sentimento: {resultado_paralelo['sentimento']}")
print(f"🔑 Palavras-chave: {resultado_paralelo['palavras_chave']}")
print(f"📌 Tópico: {resultado_paralelo['topico']}")
print(f"⏱️ Tempo de processamento: {end_time - start_time:.2f} segundos")
print("=" * 60)
print("\n🚀 Múltiplas análises executadas simultaneamente!")

## 🎛️ Chains Condicionais - Tomando Decisões

Agora vamos criar uma chain que toma decisões baseadas no resultado anterior!

É como um **GPS inteligente**: se o trânsito tá ruim, ele muda a rota automaticamente.

In [ ]:
# Criando uma chain condicional
# Se sentimento negativo → gera resposta empática
# Se sentimento positivo → gera parabéns
# Se neutro → pede mais informações

def criar_resposta_condicional(resultado_analise):
    """Cria uma resposta baseada no sentimento detectado"""
    
    sentimento = resultado_analise["sentimento"].lower()
    
    if "negativ" in sentimento:
        template = ChatPromptTemplate.from_messages([
            ("system", "Você é um conselheiro empático. Ofereça apoio e sugestões construtivas."),
            ("human", "A pessoa expressou sentimentos negativos sobre: {topico}. Como posso ajudar?")
        ])
    elif "positiv" in sentimento:
        template = ChatPromptTemplate.from_messages([
            ("system", "Você é entusiasta e motivador. Celebre o sucesso da pessoa."),
            ("human", "A pessoa está feliz com: {topico}. Celebre com ela!")
        ])
    else:
        template = ChatPromptTemplate.from_messages([
            ("system", "Você é curioso e quer entender melhor. Faça perguntas interessantes."),
            ("human", "A pessoa falou sobre: {topico}. Que perguntas interessantes posso fazer?")
        ])
    
    return template | llm | StrOutputParser()

# Função que orquestra a chain condicional
def chain_condicional_completa(input_data):
    """Executa análise e resposta condicional"""
    
    # Primeira etapa: análise paralela
    analise = chain_paralela.invoke(input_data)
    
    # Segunda etapa: resposta condicional
    resposta_chain = criar_resposta_condicional(analise)
    resposta = resposta_chain.invoke({"topico": analise["topico"]})
    
    return {
        "analise": analise,
        "resposta_personalizada": resposta
    }

print("🧠 Chain condicional criada!")
print("🔀 Fluxo: Análise → Decisão → Resposta Personalizada")

In [ ]:
# Testando a chain condicional com diferentes tipos de texto

textos_teste = [
    {
        "nome": "Texto Positivo",
        "texto": "Acabei de ser promovido no trabalho! Estou muito feliz e animado com os novos desafios."
    },
    {
        "nome": "Texto Negativo", 
        "texto": "Estou passando por um momento difícil. O trabalho está estressante e me sinto sobrecarregado."
    },
    {
        "nome": "Texto Neutro",
        "texto": "Hoje estudei sobre inteligência artificial. É um tópico interessante com várias aplicações."
    }
]

print("🎭 TESTANDO CHAIN CONDICIONAL COM DIFERENTES SENTIMENTOS:")
print("=" * 70)

for teste in textos_teste:
    print(f"\n📝 {teste['nome']}:")
    print(f"Texto: {teste['texto'][:50]}...")
    
    resultado = chain_condicional_completa({"texto": teste["texto"]})
    
    print(f"😊 Sentimento detectado: {resultado['analise']['sentimento']}")
    print(f"🤖 Resposta personalizada: {resultado['resposta_personalizada'][:100]}...")
    print("-" * 50)

print("\n✨ Liiindo! A chain se adaptou a cada tipo de sentimento!")

## 📈 Métricas e Performance de Chains

Como bons desenvolvedores, precisamos medir a performance das nossas chains!

In [ ]:
# Criando um sistema de benchmark para chains
import time
from datetime import datetime

class ChainBenchmark:
    """Classe para fazer benchmark de chains"""
    
    def __init__(self):
        self.resultados = []
    
    def executar_benchmark(self, chain, input_data, nome_chain="Chain", repeticoes=3):
        """Executa benchmark de uma chain"""
        
        tempos = []
        sucessos = 0
        
        print(f"🏃‍♂️ Executando benchmark para {nome_chain}...")
        
        for i in range(repeticoes):
            try:
                start_time = time.time()
                
                if callable(chain):
                    resultado = chain(input_data)
                else:
                    resultado = chain.invoke(input_data)
                
                end_time = time.time()
                tempo_execucao = end_time - start_time
                tempos.append(tempo_execucao)
                sucessos += 1
                
                print(f"   Execução {i+1}: {tempo_execucao:.2f}s ✅")
                
            except Exception as e:
                print(f"   Execução {i+1}: ERRO - {str(e)[:50]}... ❌")
        
        # Calculando métricas
        if tempos:
            tempo_medio = sum(tempos) / len(tempos)
            tempo_min = min(tempos)
            tempo_max = max(tempos)
            taxa_sucesso = (sucessos / repeticoes) * 100
        else:
            tempo_medio = tempo_min = tempo_max = 0
            taxa_sucesso = 0
        
        resultado_benchmark = {
            "nome": nome_chain,
            "tempo_medio": tempo_medio,
            "tempo_min": tempo_min,
            "tempo_max": tempo_max,
            "taxa_sucesso": taxa_sucesso,
            "repeticoes": repeticoes,
            "timestamp": datetime.now()
        }
        
        self.resultados.append(resultado_benchmark)
        return resultado_benchmark
    
    def relatorio(self):
        """Gera relatório dos benchmarks"""
        if not self.resultados:
            print("Nenhum benchmark executado ainda!")
            return
        
        print("\n📊 RELATÓRIO DE PERFORMANCE:")
        print("=" * 60)
        
        for resultado in self.resultados:
            print(f"\n🔗 {resultado['nome']}:")
            print(f"   ⏱️ Tempo médio: {resultado['tempo_medio']:.2f}s")
            print(f"   🚀 Tempo mínimo: {resultado['tempo_min']:.2f}s")
            print(f"   🐌 Tempo máximo: {resultado['tempo_max']:.2f}s")
            print(f"   ✅ Taxa de sucesso: {resultado['taxa_sucesso']:.1f}%")

# Criando nosso benchmarker
benchmark = ChainBenchmark()
print("📊 Sistema de benchmark criado!")

In [ ]:
# Fazendo benchmark das nossas chains

texto_benchmark = "A tecnologia blockchain tem potencial para revolucionar diversos setores da economia."

# Benchmark da chain simples
benchmark.executar_benchmark(
    receita_chain, 
    {"ingrediente": "chocolate", "tipo_prato": "sobremesa"}, 
    "Chain Simples (Receita)"
)

# Benchmark da chain paralela
benchmark.executar_benchmark(
    chain_paralela, 
    {"texto": texto_benchmark}, 
    "Chain Paralela"
)

# Benchmark da chain condicional
benchmark.executar_benchmark(
    chain_condicional_completa, 
    {"texto": texto_benchmark}, 
    "Chain Condicional"
)

# Gerando relatório
benchmark.relatorio()

In [ ]:
# Visualizando a performance das chains

def visualizar_performance():
    """Cria gráfico de performance das chains"""
    
    if not benchmark.resultados:
        print("Nenhum resultado para visualizar!")
        return
    
    # Preparando dados
    nomes = [r["nome"] for r in benchmark.resultados]
    tempos_medios = [r["tempo_medio"] for r in benchmark.resultados]
    taxas_sucesso = [r["taxa_sucesso"] for r in benchmark.resultados]
    
    # Criando gráficos
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Gráfico de tempo médio
    bars1 = ax1.bar(nomes, tempos_medios, color=['skyblue', 'lightgreen', 'salmon'])
    ax1.set_title('⏱️ Tempo Médio de Execução', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Segundos')
    ax1.tick_params(axis='x', rotation=45)
    
    # Adicionando valores nas barras
    for bar, tempo in zip(bars1, tempos_medios):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                f'{tempo:.2f}s', ha='center', va='bottom', fontweight='bold')
    
    # Gráfico de taxa de sucesso
    bars2 = ax2.bar(nomes, taxas_sucesso, color=['gold', 'lightcoral', 'lightblue'])
    ax2.set_title('✅ Taxa de Sucesso', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Porcentagem (%)')
    ax2.set_ylim(0, 105)
    ax2.tick_params(axis='x', rotation=45)
    
    # Adicionando valores nas barras
    for bar, taxa in zip(bars2, taxas_sucesso):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{taxa:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Gráficos de performance gerados!")
    print("💡 Análise: Chains mais complexas levam mais tempo, mas oferecem mais funcionalidades!")

visualizar_performance()

## 🚀 Chains Avançadas - Preparando para o Futuro

Agora que você já manja o básico, vamos ver algumas técnicas avançadas que vão ser úteis nos próximos módulos!

### Memory em Chains
Imagina uma chain que **lembra** das conversas anteriores! (Módulo 5 chegando! 🧠)

### RAG Chains  
Chains que buscam informações em documentos antes de responder! (Módulos 6-8! 📚)

### Agent Chains
Chains que tomam decisões sobre quais tools usar! (Módulo 9! 🤖)

**🎯 Dica do Pedro**: Tudo que vimos aqui é a base para essas funcionalidades mais avançadas!

In [ ]:
# Preview: Chain com "memória" simples
# (Vamos aprofundar no módulo 5!)

class ChainComMemoria:
    """Chain simples com memória de contexto"""
    
    def __init__(self, llm):
        self.llm = llm
        self.historico = []  # Nossa "memória" simples
        
        # Template que inclui histórico
        self.template = ChatPromptTemplate.from_messages([
            ("system", "Você é um assistente que lembra de conversas anteriores."),
            ("human", "Histórico: {historico}\n\nPergunta atual: {pergunta}")
        ])
        
        self.chain = self.template | self.llm | StrOutputParser()
    
    def conversar(self, pergunta):
        """Conversa mantendo contexto"""
        
        # Preparando histórico
        historico_str = "\n".join(self.historico[-3:])  # Últimas 3 interações
        
        # Executando chain
        resposta = self.chain.invoke({
            "historico": historico_str,
            "pergunta": pergunta
        })
        
        # Salvando na memória
        self.historico.append(f"Humano: {pergunta}")
        self.historico.append(f"Assistente: {resposta}")
        
        return resposta
    
    def limpar_memoria(self):
        """Limpa o histórico"""
        self.historico = []
        print("🧠 Memória limpa!")

# Criando chain com memória
chain_memoria = ChainComMemoria(llm)
print("🧠 Chain com memória criada!")
print("💡 Preview do Módulo 5: Memory Systems")

In [ ]:
# Testando a chain com memória

print("🗣️ TESTANDO CHAIN COM MEMÓRIA:")
print("=" * 50)

# Primeira pergunta
resposta1 = chain_memoria.conversar("Meu nome é João e eu gosto de pizza")
print(f"👤 João: Meu nome é João e eu gosto de pizza")
print(f"🤖 Bot: {resposta1}")
print()

# Segunda pergunta - testando se lembra
resposta2 = chain_memoria.conversar("Qual é o meu nome?")
print(f"👤 João: Qual é o meu nome?")
print(f"🤖 Bot: {resposta2}")
print()

# Terceira pergunta - testando se lembra da preferência
resposta3 = chain_memoria.conversar("Do que eu gosto?")
print(f"👤 João: Do que eu gosto?")
print(f"🤖 Bot: {resposta3}")
print()

print("✨ A chain lembrou das informações anteriores!")
print(f"📝 Histórico atual: {len(chain_memoria.historico)} entradas")

## 💪 Exercício Prático 1: Sua Primeira Chain Personalizada

Agora é sua vez! Vamos criar uma chain que:

1. **Recebe uma notícia**
2. **Analisa se é verdadeira ou fake news**  
3. **Extrai pontos principais**
4. **Gera um resumo didático**

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-04_img_05.png)

In [ ]:
# EXERCÍCIO 1: Complete o código abaixo

# Dica: Use o que aprendemos sobre chains paralelas e sequenciais!

def criar_chain_fake_news():
    """Crie uma chain para análise de notícias"""
    
    # TODO: Crie os prompts para cada etapa
    analise_veracidade_prompt = ChatPromptTemplate.from_messages([
        ("system", "Você é um especialista em verificação de fatos. Analise se a notícia parece verdadeira, falsa ou duvidosa."),
        ("human", "Notícia: {noticia}")
    ])
    
    # TODO: Complete os outros prompts
    pontos_principais_prompt = # SEU CÓDIGO AQUI
    
    resumo_prompt = # SEU CÓDIGO AQUI
    
    # TODO: Crie as chains individuais
    # DICA: template | llm | StrOutputParser()
    
    # TODO: Combine em uma chain paralela/sequencial
    
    return # SUA CHAIN AQUI

print("📝 EXERCÍCIO 1: Complete a função acima!")
print("💡 Dica: Use chains paralelas para análises simultâneas")
print("🎯 Objetivo: Criar um verificador de notícias inteligente")

## 🎮 Exercício Prático 2: Chain de Geração de Conteúdo

Crie uma chain que gera conteúdo para redes sociais:

1. **Recebe um tópico**
2. **Gera múltiplas versões** (LinkedIn, Twitter, Instagram)
3. **Adapta o tom** para cada plataforma
4. **Sugere hashtags** relevantes

In [ ]:
# EXERCÍCIO 2: Implemente uma chain de geração de conteúdo

def criar_chain_redes_sociais():
    """Chain que adapta conteúdo para diferentes redes sociais"""
    
    # TODO: Crie prompts específicos para cada rede social
    linkedin_prompt = # SEU CÓDIGO - Tom profissional, 200-300 palavras
    twitter_prompt = # SEU CÓDIGO - Tom casual, máximo 280 caracteres  
    instagram_prompt = # SEU CÓDIGO - Tom visual, com call-to-action
    hashtags_prompt = # SEU CÓDIGO - Gera hashtags relevantes
    
    # TODO: Implemente a lógica da chain
    # DICA: Use processamento paralelo para gerar tudo simultaneamente
    
    pass

# Teste sua implementação:
# chain_social = criar_chain_redes_sociais()
# resultado = chain_social.invoke({"topico": "Inteligência Artificial no trabalho"})

print("📱 EXERCÍCIO 2: Crie um gerador de conteúdo multi-plataforma!")
print("🎯 Desafio: Adaptar automaticamente o tom e formato para cada rede")
print("💡 Dica: Cada plataforma tem suas características únicas")

## 🎯 Resumo e Próximos Passos

### O que Aprendemos Hoje:

✅ **Conceitos Fundamentais de Chains**
- O que são e por que são importantes
- Analogia da macarronada (processamento sequencial)

✅ **Tipos de Chains**
- Chains simples (linear)
- Chains paralelas (processamento simultâneo)
- Chains condicionais (tomada de decisão)

✅ **LCEL (LangChain Expression Language)**
- Sintaxe com operador `|`
- RunnablePassthrough e RunnableLambda
- Composição flexível de componentes

✅ **Performance e Benchmarking**
- Como medir tempo de execução
- Análise de taxa de sucesso
- Visualização de métricas

✅ **Preview de Funcionalidades Avançadas**
- Chains com memória (Módulo 5)
- RAG chains (Módulos 6-8)
- Agent chains (Módulo 9)

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-04_img_06.png)

In [ ]:
# Visualização final: Mapa conceitual do que aprendemos

def criar_mapa_conceitual():
    """Cria um mapa visual dos conceitos aprendidos"""
    
    fig, ax = plt.subplots(figsize=(16, 10))
    
    # Definindo conceitos e suas posições
    conceitos = {
        "CHAINS": {"pos": (8, 8), "cor": "gold", "tamanho": 1000},
        "Simples": {"pos": (4, 6), "cor": "lightblue", "tamanho": 600},
        "Paralelas": {"pos": (8, 6), "cor": "lightgreen", "tamanho": 600},
        "Condicionais": {"pos": (12, 6), "cor": "lightcoral", "tamanho": 600},
        "LCEL": {"pos": (2, 4), "cor": "orange", "tamanho": 500},
        "PromptTemplate": {"pos": (4, 4), "cor": "lavender", "tamanho": 400},
        "LLM": {"pos": (6, 4), "cor": "lightgray", "tamanho": 400},
        "OutputParser": {"pos": (8, 4), "cor": "lightpink", "tamanho": 400},
        "Performance": {"pos": (10, 4), "cor": "lightyellow", "tamanho": 400},
        "Benchmark": {"pos": (12, 4), "cor": "lightsteelblue", "tamanho": 400},
        "Memory\n(Módulo 5)": {"pos": (3, 2), "cor": "mistyrose", "tamanho": 300},
        "RAG\n(Módulos 6-8)": {"pos": (8, 2), "cor": "lightcyan", "tamanho": 300},
        "Agents\n(Módulo 9)": {"pos": (13, 2), "cor": "wheat", "tamanho": 300}
    }
    
    # Desenhando conceitos
    for nome, info in conceitos.items():
        ax.scatter(info["pos"][0], info["pos"][1], 
                  s=info["tamanho"], c=info["cor"], 
                  alpha=0.7, edgecolors='black', linewidth=2)
        ax.text(info["pos"][0], info["pos"][1], nome, 
               ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Desenhando conexões
    conexoes = [
        ("CHAINS", "Simples"),
        ("CHAINS", "Paralelas"),
        ("CHAINS", "Condicionais"),
        ("Simples", "LCEL"),
        ("Simples", "PromptTemplate"),
        ("Paralelas", "Performance"),
        ("Condicionais", "Benchmark")
    ]
    
    for origem, destino in conexoes:
        pos_origem = conceitos[origem]["pos"]
        pos_destino = conceitos[destino]["pos"]
        ax.plot([pos_origem[0], pos_destino[0]], 
                [pos_origem[1], pos_destino[1]], 
                'k--', alpha=0.3, linewidth=1)
    
    ax.set_xlim(0, 16)
    ax.set_ylim(0, 10)
    ax.set_title("🗺️ Mapa Conceitual: Chains no LangChain", 
                fontsize=18, fontweight='bold', pad=20)
    ax.axis('off')
    
    # Legenda
    ax.text(8, 0.5, "Conceitos aprendidos hoje + Preview dos próximos módulos", 
            ha='center', fontsize=12, style='italic')
    
    plt.tight_layout()
    plt.show()
    
    print("🗺️ Mapa conceitual criado!")
    print("📚 Você dominou as bases para os próximos módulos!")

criar_mapa_conceitual()

## 🚀 Preparando para o Próximo Módulo

### Módulo 5: Memory Systems 🧠

No próximo módulo vamos aprender:

- **ConversationBufferMemory**: Lembrando de tudo
- **ConversationSummaryMemory**: Resumindo histórico  
- **ConversationKGMemory**: Memória com grafos
- **Custom Memory**: Criando suas próprias soluções

**🎯 Dica do Pedro**: As chains que criamos hoje vão ganhar superpoderes com memória!

### Para Casa 📚

1. **Experimente** diferentes combinações de chains
2. **Meça performance** das suas implementações  
3. **Pense** em casos de uso do seu trabalho
4. **Complete** os exercícios se ainda não fez

---

**🎉 Parabéns!** Você completou o Módulo 4 e já manja chains como um pro!

**🔗 Bora para o próximo módulo** aprender sobre Memory Systems!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-04_img_07.png)